In [1]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Program, Task
import pandas as pd
from dataclasses import replace

from code_widget.example import CodeWidget
import json

In [2]:
# CHANGE ME!
TASK_ID = 'reachable'
AUTHOR = 'scott'

In [3]:
task = Task(
    id=TASK_ID,
    name="Reflexive-transitive closure",
    description="A path is a sequence of edges such that the target of each edge is the source of the next. \
Given a graph and a vertex v, list the vertices reachable by a path from v.",
    category="Graphs",
    plan=[
    {
        "id": "graph",
        "description": "a graph",
    }, {
        "id": "source",
        "description": "from v",
    }, {
        "id": "edge_match",
        "description": "target of each edge is the source of the next",
    }, {
        "id": "edge_sequence",
        "description": "a sequence of edges",
    }, {
        "id": "vertices",
        "description": "the vertices reachable by a path",
    }
    ],
    sample_input={
        "graph": [
            {"source": "a", "target": "b"},
            {"source": "b", "target": "b"},
            {"source": "b", "target": "c"},
            {"source": "c", "target": "f"},
            {"source": "c", "target": "b"},
            {"source": "a", "target": "d"},
            {"source": "e", "target": "a"},
        ],
        "query": [{"source": "a"}]
    },
    sample_output=["a", "b", "c", "d", "f"],
)
task.save()

prototype = Program(
    task=TASK_ID,
    author=AUTHOR,
    language=''    
)

In [9]:
q = replace(prototype,
    language='q',
    source='''reachable: {[graph; query]
  graph: (first') each graph;
  nodes:: asc distinct graph[`source], graph[`target];
  adj_list: `source xgroup graph;
  adj_matrix: {in[;x] each nodes} each nodes ,' (adj_list each nodes) `target;
  reachable: last ({x | any each (x *\: x)} \) adj_matrix;
  query_idx: nodes ? (first first query[`source]);
  nodes where reachable[query_idx]
  }''').load_plan()
q.execute(task, debug=True)
q.save()

/var/folders/6p/3bzglbgn2ts3v16zf_27zq3r0000gn/T/tmpqplluty3
graph:("**"; enlist ",") 0:`:/var/folders/6p/3bzglbgn2ts3v16zf_27zq3r0000gn/T/tmpqplluty3/graph.csv
query:("**"; enlist ",") 0:`:/var/folders/6p/3bzglbgn2ts3v16zf_27zq3r0000gn/T/tmpqplluty3/query.csv
reachable: {[graph; query]
  graph: (first') each graph;
  nodes:: asc distinct graph[`source], graph[`target];
  adj_list: `source xgroup graph;
  adj_matrix: {in[;x] each nodes} each nodes ,' (adj_list each nodes) `target;
  reachable: last ({x | any each (x *\: x)} \) adj_matrix;
  query_idx: nodes ? (first first query[`source]);
  nodes where reachable[query_idx]
  }

output: reachable[graph; query]
output: ([] c:output)
output: (asc cols output) xcols output
save `:output.csv



In [136]:
datalog = replace(prototype,
    language='datalog',
    source='''.decl path(x: symbol, y: symbol)
path(x, y) :- graph(x, y).
path(x, y) :- graph(x, z), path(z, y).
reachable(source) :- query(source).
reachable(x) :- query(source), path(source, x).''').load_plan()
datalog.execute(task)
datalog.save()

In [141]:
sql = replace(prototype,
    language='sql',
    source='''WITH RECURSIVE
closure(source, target) AS (
  SELECT source, source FROM graph
  UNION
  SELECT source, target FROM graph
  UNION
  SELECT edge.source, path.target
  FROM closure as path JOIN graph as edge
  ON edge.target = path.source
)
SELECT S.target FROM closure as S
JOIN query ON S.source = query.source''').load_plan()
sql.execute(task)
sql.save()

In [142]:
python_imp = replace(prototype,
    language='python-imperative',
    implementation='',
    source='''def reachable(graph, query):
  adjacency_list = defaultdict(list)
  for edge in graph:
    adjacency_list[edge["source"]].append(edge["target"])
  source = query[0]["source"]

  visited = set()
  to_visit = set([source])
    
  while len(to_visit) > 0:
    current = to_visit.pop()
    if current in visited:
      continue
    for neighbor in adjacency_list[current]:
      to_visit.add(neighbor)
    visited.add(current)
            
  return list(visited)''')
python_imp.execute(task)
python_imp.save()

In [143]:
python_fun = replace(prototype,
    language='python-functional',
    implementation='',
    source='''def reachable(graph, query):
  def step(visited):
    frontier = set([
      edge["target"]
      for vertex in visited
      for edge in graph
      if vertex == edge["source"]
    ])
    return frontier.union(visited)

  def fix(f, x):
    next = f(x)
    return x if next == x else fix(f, next)

  source = query[0]["source"]
  return list(fix(step, set([source])))''')
python_fun.execute(task)
python_fun.save()

In [144]:
datalog.widget(task)

Output()

CodeWidget(program='{"task": "reachable", "language": "datalog", "plan": {"graph": [], "source": [{"line": 4, …

In [145]:
python_imp.widget(task)

Output()

CodeWidget(program='{"task": "reachable", "language": "python-imperative", "plan": {}, "source": "def reachabl…

In [146]:
python_fun.widget(task)

Output()

CodeWidget(program='{"task": "reachable", "language": "python-functional", "plan": {}, "source": "def reachabl…

In [147]:
sql.widget(task)

Output()

CodeWidget(program='{"task": "reachable", "language": "sql", "plan": {"step": [{"line": 2, "start": 0, "end": …

In [64]:
# todo?
pandas = replace(prototype,
    language='python-pandas',
    implementation='',
    source='''def scc(graph):
    
''')
pandas.execute(task)
pandas.save()

SyntaxError: unexpected EOF while parsing (<string>, line 6)